#| EfficientNet - DEMO |
Implémentation de ce [paper](https://arxiv.org/pdf/1905.11946.pdf).
<br/>
<br/>

---
###Mode d'emploi:
 - Tout d'abord, appuyer sur le bouton `▶` s'affichant à côté des cellules en dessous de 'Code principale'.
 - Puis remplir les champs comme décrit dans la cellule 'DEMO'.
 - Enfin, pour lancer la prédiction du model appuyer sur le bouton `▶` s'affichant à côté des champs.

##Code principale

In [ ]:
import re
from math import ceil
import collections
from functools import partial
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import model_zoo

import collections

PATH = {
    'efficientnet-b0': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b0.pth',
    'efficientnet-b1': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b1.pth',
    'efficientnet-b2': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b2.pth',
    'efficientnet-b3': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b3.pth',
    'efficientnet-b4': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b4.pth',
    'efficientnet-b5': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b5.pth',
    'efficientnet-b6': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b6.pth',
    'efficientnet-b7': 'https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b7.pth',
}

def effnet_coef(version):
    """
    Renvoie les coefficients spécifiques du model (width, depth, res, dropout_rate)
    
    Args:
     - version (str): nom de la version du model
    """
    coefs = {
        # Coefficients:   width,depth,res,dropout_rate
        'efficientnet-b0': (1.0, 1.0, 224, 0.2),
        'efficientnet-b1': (1.0, 1.1, 240, 0.2),
        'efficientnet-b2': (1.1, 1.2, 260, 0.3),
        'efficientnet-b3': (1.2, 1.4, 300, 0.3),
        'efficientnet-b4': (1.4, 1.8, 380, 0.4),
        'efficientnet-b5': (1.6, 2.2, 456, 0.4),
        'efficientnet-b6': (1.8, 2.6, 528, 0.5),
        'efficientnet-b7': (2.0, 3.1, 600, 0.5),
    }
    return coefs[version]

def effnet_params(width_coefficient, depth_coefficient, image_size, dropout_rate, n_classes=1000, include_top=True):
    """
    Renvoie les paramètres globaux et paramètres des blocks du model (BLOCKS_PARAMS, global_params)
    
    Args:
     - width_coefficient (int)
     - depth_coefficient (int)
     - image_size (int)
     - dropout_rate (float)
     - num_classes (int)
     - include_top (bool)
    """

    # paramètres des blocks du model efficientnet-b0
    # il sera modifié dans la Class EfficientNet pour fit avec les autres versions
    block = collections.namedtuple('BlockArgs', [
        'num_repeat', 'kernel_size', 'stride', 'expand_ratio',
        'input_filters', 'output_filters', 'se_ratio', 'id_skip']
    )
    BLOCKS_PARAMS = [
        block(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, id_skip=True),
        block(num_repeat=2, kernel_size=3, stride=[2], expand_ratio=6, input_filters=16, output_filters=24, se_ratio=0.25, id_skip=True),
        block(num_repeat=2, kernel_size=5, stride=[2], expand_ratio=6, input_filters=24, output_filters=40, se_ratio=0.25, id_skip=True),
        block(num_repeat=3, kernel_size=3, stride=[2], expand_ratio=6, input_filters=40, output_filters=80, se_ratio=0.25, id_skip=True),
        block(num_repeat=3, kernel_size=5, stride=[1], expand_ratio=6, input_filters=80, output_filters=112, se_ratio=0.25, id_skip=True),
        block(num_repeat=4, kernel_size=5, stride=[2], expand_ratio=6, input_filters=112, output_filters=192, se_ratio=0.25, id_skip=True),
        block(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=6, input_filters=192, output_filters=320, se_ratio=0.25, id_skip=True),
    ]

    global_params = {
        'width_coefficient': width_coefficient,
        'depth_coefficient': depth_coefficient,
        'image_size': image_size,
        'dropout_rate': dropout_rate,
        'num_classes': n_classes,
        'drop_connect_rate': 0.2,
        'width_divisor': 8,
        'include_top': include_top
    }

    return BLOCKS_PARAMS, global_params

def get_model_params(model_version, n_classes=1000):
    """
    Renvoie les paramètres globaux et paramètres des blocks du model d'une version donnée (blocks_params, global_params)

    Args:
     - model_version (str): nom de la version du model (ex: efficientnet-b3)
    """
    w, d, s, p = effnet_coef(model_version)
    blocks_params, global_params = effnet_params(width_coefficient=w, depth_coefficient=d, dropout_rate=p, image_size=s, n_classes=n_classes)

    return blocks_params, global_params

def calculer_width(in_channels, global_params):
    """
    Calcule le nombre de channels en sortie de la Conv 
    (Calcule la width du model)
    
    Args:
     - in_channels: nombre de channels en entrée
     - global_params: parametres globaux du NN

    Return: nombre de channels en sortie
    """
    multiplicateur = global_params['width_coefficient'] #multiplie par le coef de largeur
    diviseur = global_params['width_divisor']

    in_channels = in_channels * multiplicateur
    out_channels = max(diviseur, int(in_channels + diviseur / 2) // diviseur * diviseur) #formule de l'implementation officielle
    if out_channels < 0.9 * in_channels: # permet de ne pas avoir moins de 90% de la largeur de départ
        out_channels += diviseur
    return int(out_channels)


def calculer_depth(repetition, global_params):
    """
    Calcule le nombre de layers (MBConvBlock)
    (Calcule la depth du model)
    
    Args:
     - repetition: parametre de répétition du block
     - global_params: parametres globaux du NN
    
    Return: nombre de répétition des layers
    """
    multiplicateur = global_params['depth_coefficient']
    return int(ceil(multiplicateur * repetition)) #formule de l'implemeentation officielle


def drop_connect(inputs, prob, training):
    """
    Crée un DropConnect

    Args:
     - inputs (tensor)
     - prob: probabilité déconnection
    retourne les outputs après les connections drop connect
    """
    if not training: #si pas en mode entrainement: ignorer
        return inputs

    batch = inputs.shape[0]
    prob_connection = 1 - prob
    #création tensor mask
    random_tensor = prob_connection
    random_tensor += torch.rand([batch_size, 1, 1, 1], dtype=inputs.dtype, device=inputs.device)
    mask_tensor = torch.floor(random_tensor)

    output = inputs / prob_connection * mask_tensor #application du mask sur les inputs
    return output


def calcule_taille_output_image(taille_image, stride):
    """
    Calcule la taille de l'image de sortie après application de 
    Conv2dSamePadding avec un stride (nécessite static padding donc toute meme taille d'image)

    Args:
     - taille_input_image
     - stride: valeur du stride appliqué
    """
    if taille_image is None: #si pas de taille précisé inutile de calculer
        return None 
    h, w = taille_image if isinstance(taille_image, list) or isinstance(taille_image, tuple) else (taille_image, taille_image)
    stride = stride if isinstance(stride, int) else stride[0]
    h = int(ceil(h / stride))
    w = int(ceil(w / stride))
    return [h, w]


class SwishFn(torch.autograd.Function):
    """swish activation (squelette)"""
    @staticmethod
    def forward(tensor, x):
        resultat = x * torch.sigmoid(x)
        tensor.save_for_backward(x)
        return resultat

    @staticmethod
    def backward(tensor, grad_output):
        x = tensor.saved_tensors[0]
        sigmoid_x = torch.sigmoid(x)
        return grad_output * (sigmoid_x * (1 + x * (1 - sigmoid_x)))

class Swish(nn.Module):
    """Fonction d'activation mise en avant par Google Brain ( x*sigmoid(x) )"""
    def forward(self, x):
        return SwishFn.apply(x) #methode de torch.autograd.Function (pour forward et backward)


def get_same_padding_conv2d(image_size=None):
    """
    Choisit padding static si la taille d'image est spécifiée ou padding dynamic sinon
    Renvoie la Conv2D choisi

    Args:
     - image_size (tuple)
    """
    if image_size is None:
        return Conv2dDynamicSamePadding
    else:
        return partial(Conv2dStaticSamePadding, image_size=image_size) #classe avec taille_image deja specifiée


class Conv2dDynamicSamePadding(nn.Conv2d):
    """
    Conv2D avec un SAME-padding pour des tailles d'images non spécifiées
    La valeur du padding est calculée dynamiquement puis appliquée dans le 'forward'
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
        super().__init__(in_channels, out_channels, kernel_size, stride, 0, dilation, groups, bias)
        self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2

    def forward(self, x): #(h:height, w:width) voir les implémentations
        ih, iw = x.size()[-2:]
        sh, sw = self.stride
        kh, kw = self.weight.size()[-2:]
        oh, ow = ceil(ih / sh), ceil(iw / sw) #change la taille en output en accord avec le stride
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2]) #applique le padding
        return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups) #effectue la Conv2D


class Conv2dStaticSamePadding(nn.Conv2d):
    """
    Conv2D avec un SAME-padding pour la tailles des images spécifiée
    La valeur du padding est calculée une fois dans 'init' puis appliquée dans le 'forward'
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, image_size=None, **kwargs):
        super().__init__(in_channels, out_channels, kernel_size, stride, **kwargs)
        self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2

        #calcule le padding en fonction de la taille donnée des images 
        assert image_size is not None
        ih, iw = (image_size, image_size) if isinstance(image_size, int) else image_size
        sh, sw = self.stride
        kh, kw = self.weight.size()[-2:]
        oh, ow = ceil(ih / sh), ceil(iw / sw)
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            self.static_padding = nn.ZeroPad2d((pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2))
        else:
            self.static_padding = nn.Identity() #si pas de padding à ajouter --> applique fonction qui fait rien

    def forward(self, x):
        x = self.static_padding(x)
        x = F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups) #effectuer la Conv2D
        return x


def get_same_padding_maxPool2d(image_size=None):
    """
    Choisit padding static si la taille d'image est spécifiée ou padding dynamic sinon
    Renvoie le MaxPool choisi

    Args:
     - image_size (tuple)
    """
    if image_size is None:
        return MaxPool2dDynamicSamePadding
    else:
        return partial(MaxPool2dStaticSamePadding, image_size=image_size)


class MaxPool2dDynamicSamePadding(nn.MaxPool2d):
    """
    MawPool2D avec un SAME-padding pour des tailles d'images non spécifiées
    La valeur du padding est calculée dynamiquement puis appliquée dans le 'forward'
    """
    def __init__(self, kernel_size, stride, padding=0, dilation=1, return_indices=False, ceil_mode=False):
        super().__init__(kernel_size, stride, padding, dilation, return_indices, ceil_mode)
        self.stride = [self.stride] * 2 if isinstance(self.stride, int) else self.stride
        self.kernel_size = [self.kernel_size] * 2 if isinstance(self.kernel_size, int) else self.kernel_size
        self.dilation = [self.dilation] * 2 if isinstance(self.dilation, int) else self.dilation

    def forward(self, x): #Voir les implémentations
        ih, iw = x.size()[-2:]
        kh, kw = self.kernel_size
        sh, sw = self.stride
        oh, ow = ceil(ih / sh), ceil(iw / sw)
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2])
        return F.max_pool2d(x, self.kernel_size, self.stride, self.padding,
                            self.dilation, self.ceil_mode, self.return_indices)

class MaxPool2dStaticSamePadding(nn.MaxPool2d):
    """
    MawPool2D avec un SAME-padding pour la tailles des images spécifiée
    La valeur du padding est calculée une fois dans 'init' puis appliquée dans le 'forward'
    """

    def __init__(self, kernel_size, stride, image_size=None, **kwargs):
        super().__init__(kernel_size, stride, **kwargs)
        self.stride = [self.stride] * 2 if isinstance(self.stride, int) else self.stride
        self.kernel_size = [self.kernel_size] * 2 if isinstance(self.kernel_size, int) else self.kernel_size
        self.dilation = [self.dilation] * 2 if isinstance(self.dilation, int) else self.dilation

        #calcule le padding en fonction de la taille donnée des images 
        assert image_size is not None
        ih, iw = (image_size, image_size) if isinstance(image_size, int) else image_size
        kh, kw = self.kernel_size
        sh, sw = self.stride
        oh, ow = math.ceil(ih / sh), math.ceil(iw / sw)
        pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
        pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
        if pad_h > 0 or pad_w > 0:
            self.static_padding = nn.ZeroPad2d((pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2))
        else:
            self.static_padding = nn.Identity()

    def forward(self, x):
        x = self.static_padding(x)
        x = F.max_pool2d(x, self.kernel_size, self.stride, self.padding,
                         self.dilation, self.ceil_mode, self.return_indices)
        return x

#NN

class MBConvBlock(nn.Module):
    """
    Mobile Inverted Residual Bottleneck Block
    Block tiré de MobileNet v3 (https://arxiv.org/abs/1905.02244)
    """
    def __init__(self, block_params, global_params, image_size=None):
        super().__init__()
        self._block_params = block_params
        #parametres du batch norm
        self._bn_mom = 0.01
        self._bn_eps = 1e-3

        self._se = (self._block_params.se_ratio is not None) and (0 < self._block_params.se_ratio <= 1) #Bool
        self.id_skip = block_params.id_skip  #drop connect ou pas

        # phase d'extansion (Inverted Bottleneck)
        inp = self._block_params.input_filters  #nb de input channels
        output = self._block_params.input_filters * self._block_params.expand_ratio  #nb de output channels (après expension/aggrandissement de la width)
        if self._block_params.expand_ratio != 1:
            Conv2d = get_same_padding_conv2d(image_size)
            self._expand_conv = Conv2d(in_channels=inp, out_channels=output, kernel_size=1, bias=False)
            self._bn0 = nn.BatchNorm2d(num_features=output, momentum=self._bn_mom, eps=self._bn_eps)

        # phase de convolution en profondeur
        k = self._block_params.kernel_size
        s = self._block_params.stride
        Conv2d = get_same_padding_conv2d(image_size)
        self._depthwise_conv = Conv2d(in_channels=output, out_channels=output, groups=output, kernel_size=k, stride=s, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=output, momentum=self._bn_mom, eps=self._bn_eps)
        image_size = calcule_taille_output_image(image_size, s)

        # Squeeze et Excitation layer
        if self._se:
            Conv2d = get_same_padding_conv2d(image_size=(1, 1))
            num_squeezed_channels = max(1, int(self._block_params.input_filters * self._block_params.se_ratio)) #!
            self._se_reduce = Conv2d(in_channels=output, out_channels=num_squeezed_channels, kernel_size=1)
            self._se_expand = Conv2d(in_channels=num_squeezed_channels, out_channels=output, kernel_size=1)

        # phase de convolution finale
        final_output = self._block_params.output_filters
        Conv2d = get_same_padding_conv2d(image_size)
        self._project_conv = Conv2d(in_channels=output, out_channels=final_output, kernel_size=1, bias=False)
        self._bn2 = nn.BatchNorm2d(num_features=final_output, momentum=self._bn_mom, eps=self._bn_eps)
        self._swish = Swish()

    def forward(self, inputs, drop_connect_rate=None):
        """drop_connect_rate (float (entre 0 et 1)"""

        # conv d'extansion (Inverted Bottleneck) et en profondeur
        x = inputs
        if self._block_params.expand_ratio != 1:
            x = self._expand_conv(inputs)
            x = self._bn0(x)
            x = self._swish(x)

        x = self._depthwise_conv(x)
        x = self._bn1(x)
        x = self._swish(x)

        # Squeeze et Excitation
        if self._se:
            x_squeezed = F.adaptive_avg_pool2d(x, 1)
            x_squeezed = self._se_reduce(x_squeezed)
            x_squeezed = self._swish(x_squeezed)
            x_squeezed = self._se_expand(x_squeezed)
            x = torch.sigmoid(x_squeezed) * x #Swish

        # convolution finale
        x = self._project_conv(x)
        x = self._bn2(x)

        #application du drop connect et des skip connections
        input_filters, output_filters = self._block_params.input_filters, self._block_params.output_filters
        if self.id_skip and self._block_params.stride == 1 and input_filters == output_filters:
            if drop_connect_rate:
                x = drop_connect(x, prob=drop_connect_rate, training=self.training)

            x = x + inputs  # skip connection
        return x


class EfficientNet(nn.Module):
    """EfficientNet model <3"""

    def __init__(self, blocks_params=None, global_params=None):
        super().__init__()
        assert isinstance(blocks_params, list), 'blocks_params doit etre une liste'
        self._global_params = global_params
        self._blocks_params = blocks_params

        #parametres du batch norm
        bn_mom = 0.01
        bn_eps = 1e-3

        #sélection conv static ou dynamique
        image_size = global_params['image_size']
        Conv2d = get_same_padding_conv2d(image_size)

        # stem (tronc)
        in_channels = 3  #rgb
        out_channels = calculer_width(32, self._global_params)  # nb de output channels
        self._conv_stem = Conv2d(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        self._bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)
        image_size = calcule_taille_output_image(image_size, 2) #reset de image size

        # MBConvBlock
        self._blocks = nn.ModuleList([])
        for block_params in self._blocks_params:
            # calculer les inputs et outputs filters du block en fonction de ses parametres
            #block_params['input_filters'] = calculer_width(block_params['input_filters'], self._global_params)
            #block_params['output_filters'] = calculer_width(block_params['output_filters'], self._global_params)
            #block_params['num_repeat'] = calculer_depth(block_params['num_repeat'], self._global_params)
            block_params = block_params._replace(
                input_filters=calculer_width(block_params.input_filters, self._global_params),
                output_filters=calculer_width(block_params.output_filters, self._global_params),
                num_repeat = calculer_depth(block_params.num_repeat, self._global_params)
            )

            # 1er block doit faire attention à l'augmentation de stride et de filter size
            self._blocks.append(MBConvBlock(block_params, self._global_params, image_size=image_size))
            image_size = calcule_taille_output_image(image_size, block_params.stride) #re-reset de image size
            if block_params.num_repeat > 1: #modifier block_params pour garder la meme output size à travers les layers
                #block_params['input_filters'] = block_params['output_filters']
                #block_params['stride'] = 1
                block_params = block_params._replace(
                    input_filters=block_params.output_filters,
                    stride=1
                )

            for _ in range(block_params.num_repeat - 1): #car déjà ajouté le 1er
                self._blocks.append(MBConvBlock(block_params, self._global_params, image_size=image_size))

        # Head
        in_channels = block_params.output_filters  # output du block final
        out_channels = calculer_width(1280, self._global_params)
        Conv2d = get_same_padding_conv2d(image_size)
        self._conv_head = Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Layer fonctionnel
        self._avg_pooling = nn.AdaptiveAvgPool2d(1)
        self._dropout = nn.Dropout(self._global_params['dropout_rate'])
        self._fc = nn.Linear(out_channels, self._global_params['num_classes'])
        self._swish = Swish()

    def extract_features(self, inputs):
        """
        Utilise layers de convolution pour extraire des features
        Return: l'output de la dernière convolution du model

        Args:
         - inputs (tensor): tensor de l'image
        """
        # Stem
        x = self._swish(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for i, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params['drop_connect_rate']
            if drop_connect_rate:
                drop_connect_rate *= float(i) / len(self._blocks) #repartition du drop connect rate sur l'ensemble des blocks
            x = block(x, drop_connect_rate=drop_connect_rate)

        # Head
        x = self._swish(self._bn1(self._conv_head(x)))

        return x

    def forward(self, inputs):
        """
        (Utilise extract_features pour faire le forward des Convs)
        retourne l'output du model
        """
        # forward des layers de convolution 
        x = self.extract_features(inputs)
        # Pooling et final linear layer
        x = self._avg_pooling(x)
        if self._global_params['include_top']:
            x = x.flatten(start_dim=1)
            x = self._dropout(x)
            x = self._fc(x)
        return x

    @classmethod
    def from_name(cls, version, in_channels=3, n_classes=1000):
        """
        Crée le model efficientnet correspondant à la version donnée

        Args:
         - version (str): nom de la version du model
         - in_channels (int):nombre de channel de l'image d'entrée (rgb --> 3)
        """
        blocks_params, global_params = get_model_params(version, n_classes=n_classes)
        model = cls(blocks_params, global_params)
        model._change_in_channels(in_channels)
        return model

    @classmethod
    def from_pretrained(cls, version, in_channels=3, n_classes=1000):
        """
        Crée le model efficientnet correspondant à la version donnée
        et lui load les weights pré entrainé sur ImageNet

        Args:
         - version (str): nom de la version du model
         - in_channels (int):nombre de channel de l'image d'entrée (rgb --> 3)
         - n_classes (int): nombre de classes pour la classification
        """
        model = cls.from_name(version, n_classes=n_classes)
        cls.load_weights(model, PATH[version], fc=(n_classes == 1000))
        model._change_in_channels(in_channels)
        return model

    @staticmethod
    def load_weights(model, path, fc=True):
        """
        load les weights pré-entrainés du fichier du path
        
        Args:
            model: le model efficientnet a qui on charge les weights
            path: path du fichier des weights
            fc: charger les weights des couches fonctionnelles (nn.Linear)
        """
        #state_dict = torch.load(path)
        print(f"Chargement de la sauvegarde du model depuis mon github à {path}")

        state_dict = model_zoo.load_url(path)

        if fc:
            _ = model.load_state_dict(state_dict)
            assert not _.missing_keys, 'KEYS MANQUANTES PENDANT CHARGEMENT DES WEIGHTS INCOMPLET: {}'.format(ret.missing_keys)
        else:
            state_dict.pop('_fc.weight')
            state_dict.pop('_fc.bias')
            _ = model.load_state_dict(state_dict)

        assert not _.unexpected_keys, 'KEYS PAS ATTENDUS PENDANT CHARGEMENT DES WEIGHTS INCOMPLET: {}'.format(ret.unexpected_keys)

        print(f'Weights chargés avec succès depuis {path}')
    
    def _change_in_channels(self, in_channels):
        """Adjust model's first convolution layer to in_channels, if in_channels not equals 3.
        si in_channels diff de 3, change le 1er layer de convolution pour le fit
        """
        if in_channels != 3:
            Conv2d = get_same_padding_conv2d(self._global_params['image_size'])
            out_channels = calculer_width(32, self._global_params)
            self._conv_stem = Conv2d(in_channels, out_channels, kernel_size=3, stride=2, bias=False)

In [ ]:
###################################################################################################
####################################### DEMO - EXPLOITATION #######################################
###################################################################################################

import os
from PIL import Image
import torch
import torchvision

class Predictions:
    def __init__(self, version):
        self.version = version
        self.model = EfficientNet.from_pretrained(version)
        
        self.tfms = torchvision.transforms.Compose([
            torchvision.transforms.Resize(effnet_coef(version)[2]),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), #ImageNet
            #val FlowerNet [0.287, 0.377, 0.436], [0.082, 0.060, 0.060]
        ])
        self.classes = ['tench, Tinca tinca', 'goldfish, Carassius auratus', 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias', 'tiger shark, Galeocerdo cuvieri', 'hammerhead, hammerhead shark', 'electric ray, crampfish, numbfish, torpedo', 'stingray', 'cock', 'hen', 'ostrich, Struthio camelus', 'brambling, Fringilla montifringilla', 'goldfinch, Carduelis carduelis', 'house finch, linnet, Carpodacus mexicanus', 'junco, snowbird', 'indigo bunting, indigo finch, indigo bird, Passerina cyanea', 'robin, American robin, Turdus migratorius', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel, dipper', 'kite', 'bald eagle, American eagle, Haliaeetus leucocephalus', 'vulture', 'great grey owl, great gray owl, Strix nebulosa', 'European fire salamander, Salamandra salamandra', 'common newt, Triturus vulgaris', 'eft', 'spotted salamander, Ambystoma maculatum', 'axolotl, mud puppy, Ambystoma mexicanum', 'bullfrog, Rana catesbeiana', 'tree frog, tree-frog', 'tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui', 'loggerhead, loggerhead turtle, Caretta caretta', 'leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea', 'mud turtle', 'terrapin', 'box turtle, box tortoise', 'banded gecko', 'common iguana, iguana, Iguana iguana', 'American chameleon, anole, Anolis carolinensis', 'whiptail, whiptail lizard', 'agama', 'frilled lizard, Chlamydosaurus kingi', 'alligator lizard', 'Gila monster, Heloderma suspectum', 'green lizard, Lacerta viridis', 'African chameleon, Chamaeleo chamaeleon', 'Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis', 'African crocodile, Nile crocodile, Crocodylus niloticus', 'American alligator, Alligator mississipiensis', 'triceratops', 'thunder snake, worm snake, Carphophis amoenus', 'ringneck snake, ring-necked snake, ring snake', 'hognose snake, puff adder, sand viper', 'green snake, grass snake', 'king snake, kingsnake', 'garter snake, grass snake', 'water snake', 'vine snake', 'night snake, Hypsiglena torquata', 'boa constrictor, Constrictor constrictor', 'rock python, rock snake, Python sebae', 'Indian cobra, Naja naja', 'green mamba', 'sea snake', 'horned viper, cerastes, sand viper, horned asp, Cerastes cornutus', 'diamondback, diamondback rattlesnake, Crotalus adamanteus', 'sidewinder, horned rattlesnake, Crotalus cerastes', 'trilobite', 'harvestman, daddy longlegs, Phalangium opilio', 'scorpion', 'black and gold garden spider, Argiope aurantia', 'barn spider, Araneus cavaticus', 'garden spider, Aranea diademata', 'black widow, Latrodectus mactans', 'tarantula', 'wolf spider, hunting spider', 'tick', 'centipede', 'black grouse', 'ptarmigan', 'ruffed grouse, partridge, Bonasa umbellus', 'prairie chicken, prairie grouse, prairie fowl', 'peacock', 'quail', 'partridge', 'African grey, African gray, Psittacus erithacus', 'macaw', 'sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita', 'lorikeet', 'coucal', 'bee eater', 'hornbill', 'hummingbird', 'jacamar', 'toucan', 'drake', 'red-breasted merganser, Mergus serrator', 'goose', 'black swan, Cygnus atratus', 'tusker', 'echidna, spiny anteater, anteater', 'platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus', 'wallaby, brush kangaroo', 'koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus', 'wombat', 'jellyfish', 'sea anemone, anemone', 'brain coral', 'flatworm, platyhelminth', 'nematode, nematode worm, roundworm', 'conch', 'snail', 'slug', 'sea slug, nudibranch', 'chiton, coat-of-mail shell, sea cradle, polyplacophore', 'chambered nautilus, pearly nautilus, nautilus', 'Dungeness crab, Cancer magister', 'rock crab, Cancer irroratus', 'fiddler crab', 'king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica', 'American lobster, Northern lobster, Maine lobster, Homarus americanus', 'spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish', 'crayfish, crawfish, crawdad, crawdaddy', 'hermit crab', 'isopod', 'white stork, Ciconia ciconia', 'black stork, Ciconia nigra', 'spoonbill', 'flamingo', 'little blue heron, Egretta caerulea', 'American egret, great white heron, Egretta albus', 'bittern', 'crane', 'limpkin, Aramus pictus', 'European gallinule, Porphyrio porphyrio', 'American coot, marsh hen, mud hen, water hen, Fulica americana', 'bustard', 'ruddy turnstone, Arenaria interpres', 'red-backed sandpiper, dunlin, Erolia alpina', 'redshank, Tringa totanus', 'dowitcher', 'oystercatcher, oyster catcher', 'pelican', 'king penguin, Aptenodytes patagonica', 'albatross, mollymawk', 'grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus', 'killer whale, killer, orca, grampus, sea wolf, Orcinus orca', 'dugong, Dugong dugon', 'sea lion', 'Chihuahua', 'Japanese spaniel', 'Maltese dog, Maltese terrier, Maltese', 'Pekinese, Pekingese, Peke', 'Shih-Tzu', 'Blenheim spaniel', 'papillon', 'toy terrier', 'Rhodesian ridgeback', 'Afghan hound, Afghan', 'basset, basset hound', 'beagle', 'bloodhound, sleuthhound', 'bluetick', 'black-and-tan coonhound', 'Walker hound, Walker foxhound', 'English foxhound', 'redbone', 'borzoi, Russian wolfhound', 'Irish wolfhound', 'Italian greyhound', 'whippet', 'Ibizan hound, Ibizan Podenco', 'Norwegian elkhound, elkhound', 'otterhound, otter hound', 'Saluki, gazelle hound', 'Scottish deerhound, deerhound', 'Weimaraner', 'Staffordshire bullterrier, Staffordshire bull terrier', 'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier', 'Bedlington terrier', 'Border terrier', 'Kerry blue terrier', 'Irish terrier', 'Norfolk terrier', 'Norwich terrier', 'Yorkshire terrier', 'wire-haired fox terrier', 'Lakeland terrier', 'Sealyham terrier, Sealyham', 'Airedale, Airedale terrier', 'cairn, cairn terrier', 'Australian terrier', 'Dandie Dinmont, Dandie Dinmont terrier', 'Boston bull, Boston terrier', 'miniature schnauzer', 'giant schnauzer', 'standard schnauzer', 'Scotch terrier, Scottish terrier, Scottie', 'Tibetan terrier, chrysanthemum dog', 'silky terrier, Sydney silky', 'soft-coated wheaten terrier', 'West Highland white terrier', 'Lhasa, Lhasa apso', 'flat-coated retriever', 'curly-coated retriever', 'golden retriever', 'Labrador retriever', 'Chesapeake Bay retriever', 'German short-haired pointer', 'vizsla, Hungarian pointer', 'English setter', 'Irish setter, red setter', 'Gordon setter', 'Brittany spaniel', 'clumber, clumber spaniel', 'English springer, English springer spaniel', 'Welsh springer spaniel', 'cocker spaniel, English cocker spaniel, cocker', 'Sussex spaniel', 'Irish water spaniel', 'kuvasz', 'schipperke', 'groenendael', 'malinois', 'briard', 'kelpie', 'komondor', 'Old English sheepdog, bobtail', 'Shetland sheepdog, Shetland sheep dog, Shetland', 'collie', 'Border collie', 'Bouvier des Flandres, Bouviers des Flandres', 'Rottweiler', 'German shepherd, German shepherd dog, German police dog, alsatian', 'Doberman, Doberman pinscher', 'miniature pinscher', 'Greater Swiss Mountain dog', 'Bernese mountain dog', 'Appenzeller', 'EntleBucher', 'boxer', 'bull mastiff', 'Tibetan mastiff', 'French bulldog', 'Great Dane', 'Saint Bernard, St Bernard', 'Eskimo dog, husky', 'malamute, malemute, Alaskan malamute', 'Siberian husky', 'dalmatian, coach dog, carriage dog', 'affenpinscher, monkey pinscher, monkey dog', 'basenji', 'pug, pug-dog', 'Leonberg', 'Newfoundland, Newfoundland dog', 'Great Pyrenees', 'Samoyed, Samoyede', 'Pomeranian', 'chow, chow chow', 'keeshond', 'Brabancon griffon', 'Pembroke, Pembroke Welsh corgi', 'Cardigan, Cardigan Welsh corgi', 'toy poodle', 'miniature poodle', 'standard poodle', 'Mexican hairless', 'timber wolf, grey wolf, gray wolf, Canis lupus', 'white wolf, Arctic wolf, Canis lupus tundrarum', 'red wolf, maned wolf, Canis rufus, Canis niger', 'coyote, prairie wolf, brush wolf, Canis latrans', 'dingo, warrigal, warragal, Canis dingo', 'dhole, Cuon alpinus', 'African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus', 'hyena, hyaena', 'red fox, Vulpes vulpes', 'kit fox, Vulpes macrotis', 'Arctic fox, white fox, Alopex lagopus', 'grey fox, gray fox, Urocyon cinereoargenteus', 'tabby, tabby cat', 'tiger cat', 'Persian cat', 'Siamese cat, Siamese', 'Egyptian cat', 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 'lynx, catamount', 'leopard, Panthera pardus', 'snow leopard, ounce, Panthera uncia', 'jaguar, panther, Panthera onca, Felis onca', 'lion, king of beasts, Panthera leo', 'tiger, Panthera tigris', 'cheetah, chetah, Acinonyx jubatus', 'brown bear, bruin, Ursus arctos', 'American black bear, black bear, Ursus americanus, Euarctos americanus', 'ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus', 'sloth bear, Melursus ursinus, Ursus ursinus', 'mongoose', 'meerkat, mierkat', 'tiger beetle', 'ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle', 'ground beetle, carabid beetle', 'long-horned beetle, longicorn, longicorn beetle', 'leaf beetle, chrysomelid', 'dung beetle', 'rhinoceros beetle', 'weevil', 'fly', 'bee', 'ant, emmet, pismire', 'grasshopper, hopper', 'cricket', 'walking stick, walkingstick, stick insect', 'cockroach, roach', 'mantis, mantid', 'cicada, cicala', 'leafhopper', 'lacewing, lacewing fly', "dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk", 'damselfly', 'admiral', 'ringlet, ringlet butterfly', 'monarch, monarch butterfly, milkweed butterfly, Danaus plexippus', 'cabbage butterfly', 'sulphur butterfly, sulfur butterfly', 'lycaenid, lycaenid butterfly', 'starfish, sea star', 'sea urchin', 'sea cucumber, holothurian', 'wood rabbit, cottontail, cottontail rabbit', 'hare', 'Angora, Angora rabbit', 'hamster', 'porcupine, hedgehog', 'fox squirrel, eastern fox squirrel, Sciurus niger', 'marmot', 'beaver', 'guinea pig, Cavia cobaya', 
                        'sorrel', 'zebra', 'hog, pig, grunter, squealer, Sus scrofa', 'wild boar, boar, Sus scrofa', 'warthog', 'hippopotamus, hippo, river horse, Hippopotamus amphibius', 'ox', 'water buffalo, water ox, Asiatic buffalo, Bubalus bubalis', 'bison', 'ram, tup', 'bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis', 'ibex, Capra ibex', 'hartebeest', 'impala, Aepyceros melampus', 'gazelle', 'Arabian camel, dromedary, Camelus dromedarius', 'llama', 'weasel', 'mink', 'polecat, fitch, foulmart, foumart, Mustela putorius', 'black-footed ferret, ferret, Mustela nigripes', 'otter', 'skunk, polecat, wood pussy', 'badger', 'armadillo', 'three-toed sloth, ai, Bradypus tridactylus', 'orangutan, orang, orangutang, Pongo pygmaeus', 'gorilla, Gorilla gorilla', 'chimpanzee, chimp, Pan troglodytes', 'gibbon, Hylobates lar', 'siamang, Hylobates syndactylus, Symphalangus syndactylus', 'guenon, guenon monkey', 'patas, hussar monkey, Erythrocebus patas', 'baboon', 'macaque', 'langur', 'colobus, colobus monkey', 'proboscis monkey, Nasalis larvatus', 'marmoset', 'capuchin, ringtail, Cebus capucinus', 'howler monkey, howler', 'titi, titi monkey', 'spider monkey, Ateles geoffroyi', 'squirrel monkey, Saimiri sciureus', 'Madagascar cat, ring-tailed lemur, Lemur catta', 'indri, indris, Indri indri, Indri brevicaudatus', 'Indian elephant, Elephas maximus', 'African elephant, Loxodonta africana', 'lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens', 'giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca', 'barracouta, snoek', 'eel', 'coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch', 'rock beauty, Holocanthus tricolor', 'anemone fish', 'sturgeon', 'gar, garfish, garpike, billfish, Lepisosteus osseus', 'lionfish', 'puffer, pufferfish, blowfish, globefish', 'abacus', 'abaya', "academic gown, academic robe, judge's robe", 'accordion, piano accordion, squeeze box', 'acoustic guitar', 'aircraft carrier, carrier, flattop, attack aircraft carrier', 'airliner', 'airship, dirigible', 'altar', 'ambulance', 'amphibian, amphibious vehicle', 'analog clock', 'apiary, bee house', 'apron', 'ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin', 'assault rifle, assault gun', 'backpack, back pack, knapsack, packsack, rucksack, haversack', 'bakery, bakeshop, bakehouse', 'balance beam, beam', 'balloon', 'ballpoint, ballpoint pen, ballpen, Biro', 'Band Aid', 'bannister, banister, balustrade, balusters, handrail', 'barbell', 'banjo', 'barber chair', 'barbershop', 'barn', 'barometer', 'barrel, cask', 'barrow, garden cart, lawn cart, wheelbarrow', 'baseball', 'basketball', 'bassinet', 'bassoon', 'bathing cap, swimming cap', 'bath towel', 'bathtub, bathing tub, bath, tub', 'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon', 'beacon, lighthouse, beacon light, pharos', 'beaker', 'bearskin, busby, shako', 'beer bottle', 'beer glass', 'bell cote, bell cot', 'bib', 'bicycle-built-for-two, tandem bicycle, tandem', 'bikini, two-piece', 'binder, ring-binder', 'binoculars, field glasses, opera glasses', 'birdhouse', 'boathouse', 'bobsled, bobsleigh, bob', 'bolo tie, bolo, bola tie, bola', 'bonnet, poke bonnet', 'bookcase', 'bookshop, bookstore, bookstall', 'bottlecap', 'bow', 'bow tie, bow-tie, bowtie', 'brass, memorial tablet, plaque', 'brassiere, bra, bandeau', 'breakwater, groin, groyne, mole, bulwark, seawall, jetty', 'breastplate, aegis, egis', 'broom', 'bucket, pail', 'buckle', 'bulletproof vest', 'bullet train, bullet', 'butcher shop, meat market', 'cab, hack, taxi, taxicab', 'caldron, cauldron', 'candle, taper, wax light', 'cannon', 'canoe', 'can opener, tin opener', 'cardigan', 'car mirror', 'carousel, carrousel, merry-go-round, roundabout, whirligig', "carpenter's kit, tool kit", 'carton', 'car wheel', 'cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM', 'cassette', 'cassette player', 'castle', 'catamaran', 'CD player', 'cello, violoncello', 'cellular telephone, cellular phone, cellphone, cell, mobile phone', 'chain', 'chainlink fence', 'chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour', 'chain saw, chainsaw', 'chest', 'chiffonier, commode', 'chime, bell, gong', 'china cabinet, china closet', 'Christmas stocking', 'church, church building', 'cinema, movie theater, movie theatre, movie house, picture palace', 'cleaver, meat cleaver, chopper', 'cliff dwelling', 'cloak', 'clog, geta, patten, sabot', 'cocktail shaker', 'coffee mug', 'coffeepot', 'coil, spiral, volute, whorl, helix', 'combination lock', 'computer keyboard, keypad', 'confectionery, confectionary, candy store', 'container ship, containership, container vessel', 'convertible', 'corkscrew, bottle screw', 'cornet, horn, trumpet, trump', 'cowboy boot', 'cowboy hat, ten-gallon hat', 'cradle', 'crane', 'crash helmet', 'crate', 'crib, cot', 'Crock Pot', 'croquet ball', 'crutch', 'cuirass', 'dam, dike, dyke', 'desk', 'desktop computer', 'dial telephone, dial phone', 'diaper, nappy, napkin', 'digital clock', 'digital watch', 'dining table, board', 'dishrag, dishcloth', 'dishwasher, dish washer, dishwashing machine', 'disk brake, disc brake', 'dock, dockage, docking facility', 'dogsled, dog sled, dog sleigh', 'dome', 'doormat, welcome mat', 'drilling platform, offshore rig', 'drum, membranophone, tympan', 'drumstick', 'dumbbell', 'Dutch oven', 'electric fan, blower', 'electric guitar', 'electric locomotive', 'entertainment center', 'envelope', 'espresso maker', 'face powder', 'feather boa, boa', 'file, file cabinet, filing cabinet', 'fireboat', 'fire engine, fire truck', 'fire screen, fireguard', 'flagpole, flagstaff', 'flute, transverse flute', 'folding chair', 'football helmet', 'forklift', 'fountain', 'fountain pen', 'four-poster', 'freight car', 'French horn, horn', 'frying pan, frypan, skillet', 'fur coat', 'garbage truck, dustcart', 'gasmask, respirator, gas helmet', 'gas pump, gasoline pump, petrol pump, island dispenser', 'goblet', 'go-kart', 'golf ball', 'golfcart, golf cart', 'gondola', 'gong, tam-tam', 'gown', 'grand piano, grand', 'greenhouse, nursery, glasshouse', 'grille, radiator grille', 'grocery store, grocery, food market, market', 'guillotine', 'hair slide', 'hair spray', 'half track', 'hammer', 'hamper', 'hand blower, blow dryer, blow drier, hair dryer, hair drier', 'hand-held computer, hand-held microcomputer', 'handkerchief, hankie, hanky, hankey', 'hard disc, hard disk, fixed disk', 'harmonica, mouth organ, harp, mouth harp', 'harp', 'harvester, reaper', 'hatchet', 'holster', 'home theater, home theatre', 'honeycomb', 'hook, claw', 'hoopskirt, crinoline', 'horizontal bar, high bar', 'horse cart, horse-cart', 'hourglass', 'iPod', 'iron, smoothing iron', "jack-o'-lantern", 'jean, blue jean, denim', 'jeep, landrover', 'jersey, T-shirt, tee shirt', 'jigsaw puzzle', 'jinrikisha, ricksha, rickshaw', 'joystick', 'kimono', 'knee pad', 'knot', 'lab coat, laboratory coat', 'ladle', 'lampshade, lamp shade', 'laptop, laptop computer', 'lawn mower, mower', 'lens cap, lens cover', 'letter opener, paper knife, paperknife', 'library', 'lifeboat', 'lighter, light, igniter, ignitor', 'limousine, limo', 'liner, ocean liner', 'lipstick, lip rouge', 'Loafer', 'lotion', 'loudspeaker, speaker, speaker unit, loudspeaker system, speaker system', "loupe, jeweler's loupe", 'lumbermill, sawmill', 'magnetic compass', 'mailbag, postbag', 'mailbox, letter box', 'maillot', 'maillot, tank suit', 'manhole cover', 'maraca', 'marimba, xylophone', 'mask', 'matchstick', 'maypole', 'maze, labyrinth', 'measuring cup', 'medicine chest, medicine cabinet', 'megalith, megalithic structure', 'microphone, mike', 'microwave, microwave oven', 'military uniform', 'milk can', 'minibus', 'miniskirt, mini', 'minivan', 'missile', 'mitten', 'mixing bowl', 'mobile home, manufactured home', 'Model T', 'modem', 'monastery', 'monitor', 'moped', 'mortar', 'mortarboard', 'mosque', 'mosquito net', 'motor scooter, scooter', 'mountain bike, all-terrain bike, off-roader', 'mountain tent', 'mouse, computer mouse', 'mousetrap', 'moving van', 'muzzle', 'nail', 'neck brace', 'necklace', 'nipple', 'notebook, notebook computer', 'obelisk', 'oboe, hautboy, hautbois', 'ocarina, sweet potato', 'odometer, hodometer, mileometer, milometer', 'oil filter', 'organ, pipe organ', 'oscilloscope, scope, cathode-ray oscilloscope, CRO', 'overskirt', 'oxcart', 'oxygen mask', 'packet', 'paddle, boat paddle', 'paddlewheel, paddle wheel', 'padlock', 'paintbrush', "pajama, pyjama, pj's, jammies", 'palace', 'panpipe, pandean pipe, syrinx', 'paper towel', 'parachute, chute', 'parallel bars, bars', 'park bench', 'parking meter', 'passenger car, coach, carriage', 'patio, terrace', 'pay-phone, pay-station', 'pedestal, plinth, footstall', 'pencil box, pencil case', 'pencil sharpener', 'perfume, essence', 'Petri dish', 'photocopier', 'pick, plectrum, plectron', 'pickelhaube', 'picket fence, paling', 'pickup, pickup truck', 'pier', 'piggy bank, penny bank', 'pill bottle', 'pillow', 'ping-pong ball', 'pinwheel', 'pirate, pirate ship', 'pitcher, ewer', "plane, carpenter's plane, woodworking plane", 'planetarium', 'plastic bag', 'plate rack', 'plow, plough', "plunger, plumber's helper", 'Polaroid camera, Polaroid Land camera', 'pole', 'police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria', 'poncho', 'pool table, billiard table, snooker table', 'pop bottle, soda bottle', 'pot, flowerpot', "potter's wheel", 'power drill', 'prayer rug, prayer mat', 'printer', 'prison, prison house', 'projectile, missile', 'projector', 'puck, hockey puck', 'punching bag, punch bag, punching ball, punchball', 'purse', 'quill, quill pen', 'quilt, comforter, comfort, puff', 'racer, race car, racing car',
                        'racket, racquet', 'radiator', 'radio, wireless', 'radio telescope, radio reflector', 'rain barrel', 'recreational vehicle, RV, R.V.', 'reel', 'reflex camera', 'refrigerator, icebox', 'remote control, remote', 'restaurant, eating house, eating place, eatery', 'revolver, six-gun, six-shooter', 'rifle', 'rocking chair, rocker', 'rotisserie', 'rubber eraser, rubber, pencil eraser', 'rugby ball', 'rule, ruler', 'running shoe', 'safe', 'safety pin', 'saltshaker, salt shaker', 'sandal', 'sarong', 'sax, saxophone', 'scabbard', 'scale, weighing machine', 'school bus', 'schooner', 'scoreboard', 'screen, CRT screen', 'screw', 'screwdriver', 'seat belt, seatbelt', 'sewing machine', 'shield, buckler', 'shoe shop, shoe-shop, shoe store', 'shoji', 'shopping basket', 'shopping cart', 'shovel', 'shower cap', 'shower curtain', 'ski', 'ski mask', 'sleeping bag', 'slide rule, slipstick', 'sliding door', 'slot, one-armed bandit', 'snorkel', 'snowmobile', 'snowplow, snowplough', 'soap dispenser', 'soccer ball', 'sock', 'solar dish, solar collector, solar furnace', 'sombrero', 'soup bowl', 'space bar', 'space heater', 'space shuttle', 'spatula', 'speedboat', "spider web, spider's web", 'spindle', 'sports car, sport car', 'spotlight, spot', 'stage', 'steam locomotive', 'steel arch bridge', 'steel drum', 'stethoscope', 'stole', 'stone wall', 'stopwatch, stop watch', 'stove', 'strainer', 'streetcar, tram, tramcar, trolley, trolley car', 'stretcher', 'studio couch, day bed', 'stupa, tope', 'submarine, pigboat, sub, U-boat', 'suit, suit of clothes', 'sundial', 'sunglass', 'sunglasses, dark glasses, shades', 'sunscreen, sunblock, sun blocker', 'suspension bridge', 'swab, swob, mop', 'sweatshirt', 'swimming trunks, bathing trunks', 'swing', 'switch, electric switch, electrical switch', 'syringe', 'table lamp', 'tank, army tank, armored combat vehicle, armoured combat vehicle', 'tape player', 'teapot', 'teddy, teddy bear', 'television, television system', 'tennis ball', 'thatch, thatched roof', 'theater curtain, theatre curtain', 'thimble', 'thresher, thrasher, threshing machine', 'throne', 'tile roof', 'toaster', 'tobacco shop, tobacconist shop, tobacconist', 'toilet seat', 'torch', 'totem pole', 'tow truck, tow car, wrecker', 'toyshop', 'tractor', 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi', 'tray', 'trench coat', 'tricycle, trike, velocipede', 'trimaran', 'tripod', 'triumphal arch', 'trolleybus, trolley coach, trackless trolley', 'trombone', 'tub, vat', 'turnstile', 'typewriter keyboard', 'umbrella', 'unicycle, monocycle', 'upright, upright piano', 'vacuum, vacuum cleaner', 'vase', 'vault', 'velvet', 'vending machine', 'vestment', 'viaduct', 'violin, fiddle', 'volleyball', 'waffle iron', 'wall clock', 'wallet, billfold, notecase, pocketbook', 'wardrobe, closet, press', 'warplane, military plane', 'washbasin, handbasin, washbowl, lavabo, wash-hand basin', 'washer, automatic washer, washing machine', 'water bottle', 'water jug', 'water tower', 'whiskey jug', 'whistle', 'wig', 'window screen', 'window shade', 'Windsor tie', 'wine bottle', 'wing', 'wok', 'wooden spoon', 'wool, woolen, woollen', 'worm fence, snake fence, snake-rail fence, Virginia fence', 'wreck', 'yawl', 'yurt', 'web site, website, internet site, site', 'comic book', 'crossword puzzle, crossword', 'street sign', 'traffic light, traffic signal, stoplight', 'book jacket, dust cover, dust jacket, dust wrapper', 'menu', 'plate', 'guacamole', 'consomme', 'hot pot, hotpot', 'trifle', 'ice cream, icecream', 'ice lolly, lolly, lollipop, popsicle', 'French loaf', 'bagel, beigel', 'pretzel', 'cheeseburger', 'hotdog, hot dog, red hot', 'mashed potato', 'head cabbage', 'broccoli', 'cauliflower', 'zucchini, courgette', 'spaghetti squash', 'acorn squash', 'butternut squash', 'cucumber, cuke', 'artichoke, globe artichoke', 'bell pepper', 'cardoon', 'mushroom', 'Granny Smith', 'strawberry', 'orange', 'lemon', 'fig', 'pineapple, ananas', 'banana', 'jackfruit, jak, jack', 'custard apple', 'pomegranate', 'hay', 'carbonara', 'chocolate sauce, chocolate syrup', 'dough', 'meat loaf, meatloaf', 'pizza, pizza pie', 'potpie', 'burrito', 'red wine', 'espresso', 'cup', 'eggnog', 'alp', 'bubble', 'cliff, drop, drop-off', 'coral reef', 'geyser', 'lakeside, lakeshore', 'promontory, headland, head, foreland', 'sandbar, sand bar', 'seashore, coast, seacoast, sea-coast', 'valley, vale', 'volcano', 'ballplayer, baseball player', 'groom, bridegroom', 'scuba diver', 'rapeseed', 'daisy', "yellow lady's slipper, yellow lady-slipper, Cypripedium calceolus, Cypripedium parviflorum", 'corn', 'acorn', 'hip, rose hip, rosehip', 'buckeye, horse chestnut, conker', 'coral fungus', 'agaric', 'gyromitra', 'stinkhorn, carrion fungus', 'earthstar', 'hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa', 'bolete', 'ear, spike, capitulum', 'toilet tissue, toilet paper, bathroom tissue']

        self.downloaded_image = {}

    def predict(self, url, topk, version):
        assert ".png" in url.lower() or ".jpg" in url.lower() or ".jpeg" in url.lower(), "jpeg, jpg, png sont les seuls formats autorisés."

        if version != self.version: #si l'utilisateur change le model on load le nouveau
            from IPython.display import clear_output 
            self.model = EfficientNet.from_pretrained(version)
            self.version = version
            clear_output()

        if not url in self.downloaded_image:
            os.system(f"wget {url}")
            filename = url.split('/')[-1]
            self.downloaded_image[url] = filename
        else:
            filename = self.downloaded_image[url]
        #print(f"> {filename}\n")

        img = Image.open(filename).convert('RGB')
        img = self.tfms(img).unsqueeze(0)
        #print(f"Taille de l'image: {img.shape}") # 1, 3, 224, 224

        self.model.eval()
        with torch.no_grad():
            outputs = self.model(img)

        print("=============================================================================\n============================ PREDICTIONS DU MODEL ===========================\n=============================================================================")
        n=1
        for idx in torch.topk(outputs, k=topk).indices.squeeze(0).tolist():
            prob = torch.softmax(outputs, dim=1)[0, idx].item()
            print('{:>2}) {:<52} ({:.2f}%)'.format(n, self.classes[idx], prob*100))
            n+=1

        return outputs


demo = Predictions("efficientnet-b0")

Chargement de la sauvegarde du model depuis mon github à https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b0.pth
Weights chargés avec succès depuis https://github.com/lysandrec/Portfolio/releases/download/v1.0/efficientnet-b0.pth


##DEMO

> ATTENTION: le model utilisé ici a été entrainé à reconnaitre les objets répertoriés [ici](https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt). Par conséquent, il ne peut pas reconnaitre d'objets ne se trouvant pas dans cette liste.

Veuillez compléter les champs:
 - `Version`: Version du model (de 'b0' la plus rapide à 'b7' la plus puissante)
 - `URL`: URL de l'image sur laquelle la prédiction va etre effectuée
 - `TopK`: Permet d'afficher k meilleurs prédictions


In [ ]:
#@title {vertical-output: true}

Version = "efficientnet-b0" #@param ["efficientnet-b0", "efficientnet-b1", "efficientnet-b2", "efficientnet-b3", "efficientnet-b4", "efficientnet-b5", "efficientnet-b6", "efficientnet-b7"]
URL = "https://planspace.org/20170430-sampling_imagenet/img/n02118333_27_fox.jpg" #@param {type:"string"}
TopK = 5 #@param {type:"slider", min:1, max:10, step:1}

print()
print()
demo.predict(URL, TopK, Version)
print()

============================ PREDICTIONS DU MODEL ===========================
 1) red fox, Vulpes vulpes                               (85.35%)
 2) grey fox, gray fox, Urocyon cinereoargenteus         (2.63%)
 3) kit fox, Vulpes macrotis                             (0.97%)
 4) coyote, prairie wolf, brush wolf, Canis latrans      (0.13%)
 5) dhole, Cuon alpinus                                  (0.11%)

